In [3]:
import pandas as pd
from pathlib import Path
import json

In [6]:
log_data = pd.read_json(Path(f'../data_v2/ab_experiment_log.jsonl'), lines=True)
print(log_data)


     user_id model_type                  timestamp  \
0       1098     simple 2024-12-30 01:14:54.348548   
1       1088     simple 2024-12-30 01:14:54.888639   
2        192     simple 2024-12-30 01:14:55.407837   
3        159     simple 2024-12-30 01:14:55.910150   
4        255     simple 2024-12-30 01:14:56.432560   
..       ...        ...                        ...   
105     1040    complex 2024-12-30 01:37:36.809253   
106      967     simple 2024-12-30 01:37:37.275945   
107      756     simple 2024-12-30 01:37:37.748643   
108      872     simple 2024-12-30 01:37:38.189977   
109      619     simple 2024-12-30 01:37:38.702037   

                                    recommended_tracks   status  
0    {'id_track': '3Jl5GohfNwozDmpzmQBLDI', 'embedd...  success  
1    {'id_track': '4O1aEPLNamT9ZuClNZi2KM', 'embedd...  success  
2    {'id_track': '5ZbwJYmRtJe5Uy9nFkki8W', 'embedd...  success  
3    {'id_track': '6DKt7DVgBWadoaltUYlPz8', 'embedd...  success  
4    {'id_track': '5u

In [11]:
results = []
model_counts = {"simple": {"in_session": 0, "not_in_session": 0}, "complex": {"in_session": 0, "not_in_session": 0}}

for index, log_entry in log_data.iterrows():
    user_id = log_entry["user_id"]
    recommended_track_id = log_entry["recommended_tracks"]["id_track"]
    model_type = log_entry["model_type"]
    
    session_data = pd.read_json(Path(f'../data_v2/sessions/sessions_user_{user_id}.jsonl'), lines=True)
    
    is_in_session = any(session['track_id'] == recommended_track_id for i, session in session_data.iterrows())
    
    result = {
        "user_id": user_id,
        "recommended_track_id": recommended_track_id,
        "model_type": model_type,
        "is_in_session": is_in_session
    }
    
    results.append(result)
    
    if is_in_session:
        model_counts[model_type]["in_session"] += 1
    else:
        model_counts[model_type]["not_in_session"] += 1



In [24]:
for model_type, counts in model_counts.items():
    total_recommendations = counts["in_session"] + counts["not_in_session"]
    success_rate = (counts["in_session"] / total_recommendations) * 100 if total_recommendations > 0 else 0
    print(f"Model: {model_type}")
    print(f"Succesfully recommended tracks: {counts['in_session']}")
    print(f"Unsuccesfully recommended tracks: {counts['not_in_session']}")
    print(f"Success: {success_rate:.2f}%")



Model: simple
Succesfully recommended tracks: 3
Unsuccesfully recommended tracks: 54
Success: 5.26%
Model: complex
Succesfully recommended tracks: 6
Unsuccesfully recommended tracks: 47
Success: 11.32%
